## Other notebooks in this competition

* [Brain Tumor 3D [Training]](https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-training) 
* [Brain Tumor 3D [Inference]](https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-inference)
* [Brain Tumor 3D [EDA]](https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-eda)


## Please if this kernel is useful, <font color='red'>please upvote !!</font>

#### <font color='red'>Important Note:</font> Please don't use this notebook as a final submit, that will lead you to get a low private score because I used a 0.5 as score for all points in the private data.
### In this notebook, I just want to explain the blending and how it can improve the score.

# blending
Generally, blending uses many separate models to compute the initial prediction. the goal of mixing the predictions in some way is to achieve an even better final prediction.


# Solutions for blending

1. https://www.kaggle.com/rluethy/efficientnet3d-with-one-mri-type (Score=0.684)
1. https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-inference (Score=0.683)
1. https://www.kaggle.com/syerwin/efficientnet3d-with-one-mri-type (Score=0.674)
1. https://www.kaggle.com/hijest/1-rsna-miccai-not-smart-ensemble  (Score=0.715)



In [ ]:
import numpy as np 
import pandas as pd 
import glob
import os
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px

import plotly.graph_objects as go
from plotly.offline import iplot

In [ ]:
sub_715 = pd.read_csv('../input/brain-tumor-3d-blender-csv/RSNA_Public/715.csv')
sub_684 = pd.read_csv('../input/brain-tumor-3d-blender-csv/RSNA_Public/684.csv')
sub_683 = pd.read_csv('../input/brain-tumor-3d-blender-csv/RSNA_Public/683.csv')
sub_674 = pd.read_csv('../input/brain-tumor-3d-blender-csv/RSNA_Public/674.csv')

In [ ]:
sub_715=sub_715.sort_values(by=['BraTS21ID'], ascending=True)
sub_715=sub_715.reset_index(drop=True)
sub_715.head(3)

In [ ]:
sub_684=sub_684.sort_values(by=['BraTS21ID'], ascending=True)
sub_684=sub_684.reset_index(drop=True)
sub_684.head(3)

In [ ]:
sub_683=sub_683.sort_values(by=['BraTS21ID'], ascending=True)
sub_683=sub_683.reset_index(drop=True)
sub_683.head(3)

In [ ]:
sub_674=sub_674.sort_values(by=['BraTS21ID'], ascending=True)
sub_674=sub_674.reset_index(drop=True)
sub_674.head(3)

In [ ]:

hist_data = [sub_715.MGMT_value, sub_684.MGMT_value, sub_683.MGMT_value, sub_674.MGMT_value]
group_labels = ['sub_715', 'sub_684', 'sub_683', 'sub_674']

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False)

fig.update_layout(
    width = 700,
    height = 400
)
fig.show()

In [ ]:
# dict for the dataframes and their names
dfs = {"sub_715" : sub_715, "sub_684": sub_684, "sub_683" : sub_683, "sub_674" : sub_674}

# plot the data
fig = go.Figure()
for i in dfs:
    fig = fig.add_trace(go.Scatter(x = dfs[i]["BraTS21ID"],
                                   y = dfs[i]["MGMT_value"], 
                                   mode="markers",
                                   name = i))
    fig = fig.add_trace(
        go.Scatter(
            x=[0, 1010],
            y=[0.5, 0.5],
            mode="lines",
            line=go.scatter.Line(color="gray"),
            showlegend=False)
    )

fig.update_layout(
    width = 700,
    height = 500
)
fig.show()


In [ ]:
data=np.corrcoef(hist_data)
fig = px.imshow(data,x=group_labels,y=group_labels)
fig.show()

In [ ]:
Finalsubmission = sub_715.copy()
Finalsubmission['MGMT_value'] =\
sub_715['MGMT_value'].values*0.595+\
sub_684['MGMT_value'].values*0.2+\
sub_683['MGMT_value'].values*0.095+\
sub_674['MGMT_value'].values*0.11

In [ ]:
Finalsubmission['BraTS21ID'] = Finalsubmission['BraTS21ID'].apply(lambda x: str(x).zfill(5))

In [ ]:
Finalsubmission.head(3)

In [ ]:
Fsubmission = Finalsubmission.set_index('BraTS21ID')
FsubmissionDict = Fsubmission['MGMT_value'].to_dict()

In [ ]:
listOfStudyPaths = glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*')
listOfStudies = [eachPath.split('/')[-1] for eachPath in listOfStudyPaths]

predList = []
for eachStudy in listOfStudies:
    if eachStudy not in FsubmissionDict:
        predList.append('0.500')
    else:
        score = float(FsubmissionDict[eachStudy])
        predList.append(score)
        
submission = pd.DataFrame({'BraTS21ID':listOfStudies,'MGMT_value':predList})


In [ ]:
submission=submission.sort_values(by=['BraTS21ID'], ascending=True)
submission=submission.reset_index(drop=True)
submission.head(5)

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
plt.figure(figsize=(5, 5))
plt.hist(submission["MGMT_value"]);

# Please if this kernel is useful, <font color='red'>please upvote !!</font>